## 1. Load dataset

The example datasets contain molecule structure (SMILES) and measured bioactivity (pKi or IC50) – the higher the better. Each SMILES is converted to a Mol object in RDKit.

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem

from sklearn.metrics import r2_score, accuracy_score

In [2]:
def reg_to_clf(y):
    return np.where(np.array(y) > 6, 1, 0)

def accuracy_metric(y_true, y_pred, task=None):
    if task == "classification":
        return accuracy_score(y_true, y_pred)
    elif task == "regression":
        return r2_score(y_true, y_pred)

In [3]:
TASK = "regression"
# TASK = "classification"

In [4]:
data_train = pd.read_csv('data/CHEMBL1824/train.csv', header=None)
data_test = pd.read_csv('data/CHEMBL1824/test.csv', header=None)

In [5]:
smi_train, prop_train = data_train[0].to_list(), data_train[1].to_list()
smi_test, prop_test = data_test[0].to_list(), data_test[1].to_list()

if TASK == "classification":
    prop_train, prop_test = reg_to_clf(prop_train), reg_to_clf(prop_test)

In [6]:
mols_train, y_train = [], []
for smi, prop in zip(smi_train, prop_train):
    mol = Chem.MolFromSmiles(smi)
    if mol:
        mols_train.append(mol)
        y_train.append(prop)

In [7]:
mols_test, y_test = [], []
for smi, prop in zip(smi_test, prop_test):
    mol = Chem.MolFromSmiles(smi)
    if mol:
        mols_test.append(mol)
        y_test.append(prop)

## 1.5 Reduce the dataset size for faster pipeline (for playing around)

In [8]:
mols_train, y_train = mols_train[:80], y_train[:80]
mols_test, y_test = mols_test[:20], y_test[:20]

## 2. Conformer generation

For each molecule, an ensemble of conformers is generated. Then, molecules for which conformer generation failed are filtered out from both, the training and test set. Generated conformers can be accessed by mol.GetConformers(confID=0).

In [9]:
from qsarmil.conformer import RDKitConformerGenerator

from qsarmil.utils.logging import FailedConformer, FailedDescriptor

In [10]:
conf_gen = RDKitConformerGenerator(num_conf=10, num_cpu=40)

In [11]:
confs_train = conf_gen.run(mols_train)

tmp = [(c, y) for c, y in zip(confs_train, y_train) if not isinstance(c, FailedConformer)]
confs_train, y_train = zip(*tmp) 
confs_train, y_train = list(confs_train), list(y_train)

Generating conformers: 100%|████████████████████████████████████████████████████████████| 80/80 [00:05<00:00, 15.92it/s]


In [12]:
confs_test = conf_gen.run(mols_test)

tmp = [(c, y) for c, y in zip(confs_test, y_test) if not isinstance(c, FailedConformer)]
confs_test, y_test = zip(*tmp) 
confs_test, y_test = list(confs_test), list(y_test)

Generating conformers: 100%|████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  8.37it/s]


## 3. Descriptor calculation

Then, for each molecule with associated conformers 3D descriptors are calculated. Here, a descriptor wrapper is used, which is designed to apply descriptor calculators from external packages. The resulting descriptors are a list of 2D arrays (bags). Also, the resulting descriptors are scaled.

In [13]:
from qsarmil.descriptor.rdkit import (RDKitGEOM, 
                                      RDKitAUTOCORR, 
                                      RDKitRDF, 
                                      RDKitMORSE, 
                                      RDKitWHIM, 
                                      RDKitGETAWAY)

from molfeat.calc import Pharmacophore3D, USRDescriptors, ElectroShapeDescriptors

from qsarmil.descriptor.wrapper import DescriptorWrapper

from milearn.preprocessing import BagMinMaxScaler

In [14]:
desc_calc = DescriptorWrapper(RDKitRDF())

In [15]:
x_train = desc_calc.transform(confs_train)
x_test = desc_calc.transform(confs_test)

In [16]:
scaler = BagMinMaxScaler()

scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

## 4. Model training

In [17]:
import logging
import warnings
warnings.filterwarnings("ignore")
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
logging.getLogger("lightning").setLevel(logging.ERROR)

import time
import torch
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# MNIST dataset creation
from milearn.data.mnist import load_mnist, create_bags_or, create_bags_and, create_bags_xor, create_bags_reg

# Preprocessing
from milearn.preprocessing import BagMinMaxScaler

# Network hparams
from milearn.network.module.hopt import DEFAULT_PARAM_GRID

# MIL wrappers
from milearn.network.regressor import BagWrapperMLPNetworkRegressor, InstanceWrapperMLPNetworkRegressor
from milearn.network.classifier import BagWrapperMLPNetworkClassifier, InstanceWrapperMLPNetworkClassifier

# MIL networks
from milearn.network.regressor import (InstanceNetworkRegressor,
                                       BagNetworkRegressor,
                                       AdditiveAttentionNetworkRegressor,
                                       SelfAttentionNetworkRegressor,
                                       HopfieldAttentionNetworkRegressor,
                                       DynamicPoolingNetworkRegressor)

from milearn.network.classifier import (InstanceNetworkClassifier,
                                        BagNetworkClassifier,
                                        AdditiveAttentionNetworkClassifier,
                                        SelfAttentionNetworkClassifier,
                                        HopfieldAttentionNetworkClassifier,
                                        DynamicPoolingNetworkClassifier)

# Utils
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import train_test_split

In [18]:
regressor_list = [

        # wrapper mil networks
        ("MeanBagWrapperMLPNetworkRegressor", BagWrapperMLPNetworkRegressor(pool="mean")),
        ("MeanInstanceWrapperMLPNetworkRegressor", InstanceWrapperMLPNetworkRegressor(pool="mean")),
    
        # classic mil networks
        ("MeanBagNetworkRegressor", BagNetworkRegressor(pool="mean")),
        ("MeanInstanceNetworkRegressor", InstanceNetworkRegressor(pool="mean")),

        # attention mil networks
        ("AdditiveAttentionNetworkRegressor", AdditiveAttentionNetworkRegressor()),
        ("SelfAttentionNetworkRegressor", SelfAttentionNetworkRegressor()),
        ("HopfieldAttentionNetworkRegressor", HopfieldAttentionNetworkRegressor()),

        # other mil networks
        ("DynamicPoolingNetworkRegressor", DynamicPoolingNetworkRegressor()),
    ]

classifier_list = [

        # wrapper mil networks
        ("MeanBagWrapperMLPNetworkClassifier", BagWrapperMLPNetworkClassifier(pool="mean")),
        ("MeanInstanceWrapperMLPNetworkClassifier", InstanceWrapperMLPNetworkClassifier(pool="mean")),
    
        # classic mil networks
        ("MeanBagNetworkClassifier", BagNetworkClassifier(pool="mean")),
        ("MeanInstanceNetworkClassifier", InstanceNetworkClassifier(pool="mean")),

        # attention mil networks
        ("AdditiveAttentionNetworkClassifier", AdditiveAttentionNetworkClassifier()),
        ("SelfAttentionNetworkClassifier", SelfAttentionNetworkClassifier()),
        ("HopfieldAttentionNetworkClassifier", HopfieldAttentionNetworkClassifier()),

        # other mil networks
        ("DynamicPoolingNetworkClassifier", DynamicPoolingNetworkClassifier()),
    ]

In [22]:
if TASK == "regression":
    method_list = regressor_list
elif TASK == "classification":
    method_list = classifier_list

res_df = pd.DataFrame()
for method_name, model in method_list:
    print(method_name)

    model.hopt(x_train_scaled, y_train, param_grid=DEFAULT_PARAM_GRID, verbose=True)
    model.fit(x_train_scaled, y_train)

    if TASK == "regression":
        y_pred = model.predict(x_test_scaled)
    elif TASK == "classification":
        y_prob = model.predict(x_test_scaled)
        y_pred = np.where(y_prob > 0.5, 1, 0)
    
    res_df.loc[method_name, "ACC"] = accuracy_metric(y_test, y_pred, task=TASK)

MeanBagWrapperMLPNetworkRegressor
Optimizing hyperparameter: hidden_layer_sizes (3 options)
[1/28 |  3.6% |  0.0 min] Value: (2048, 1024, 512, 256, 128, 64), Epochs: 30, Loss: 1.9428
[2/28 |  7.1% |  0.0 min] Value: (256, 128, 64), Epochs: 51, Loss: 2.1204
[3/28 | 10.7% |  0.0 min] Value: (128,), Epochs: 47, Loss: 7.4852
Best hidden_layer_sizes = (2048, 1024, 512, 256, 128, 64), val_loss = 1.9428
Optimizing hyperparameter: activation (5 options)
[4/28 | 14.3% |  0.1 min] Value: relu, Epochs: 64, Loss: 1.6078
[5/28 | 17.9% |  0.1 min] Value: leakyrelu, Epochs: 71, Loss: 1.9903
[6/28 | 21.4% |  0.1 min] Value: gelu, Epochs: 52, Loss: 1.3541
[7/28 | 25.0% |  0.1 min] Value: elu, Epochs: 29, Loss: 2.1464
[8/28 | 28.6% |  0.1 min] Value: silu, Epochs: 34, Loss: 0.8457
Best activation = silu, val_loss = 0.8457
Optimizing hyperparameter: learning_rate (2 options)
[9/28 | 32.1% |  0.1 min] Value: 0.0001, Epochs: 55, Loss: 6.0836
[10/28 | 35.7% |  0.1 min] Value: 0.001, Epochs: 43, Loss: 0.7298

In [23]:
res_df.sort_values(by="ACC", ascending=False)

,ACC
MeanInstanceWrapperMLPNetworkRegressor,0.373213
MeanBagNetworkRegressor,0.331022
HopfieldAttentionNetworkRegressor,0.167728
SelfAttentionNetworkRegressor,0.166942
AdditiveAttentionNetworkRegressor,0.027758
MeanInstanceNetworkRegressor,0.014949
MeanBagWrapperMLPNetworkRegressor,-0.014890
DynamicPoolingNetworkRegressor,-0.090924
